# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,naze,28.3667,129.4833,23.51,50,41,4.48,JP,32400
1,1,port-aux-francais,-49.3500,70.2167,2.17,80,6,7.18,TF,18000
2,2,blackmans bay,-43.0167,147.3167,12.63,58,23,2.77,AU,39600
3,3,riberalta,-10.9833,-66.1000,27.99,60,100,0.92,BO,-14400
4,4,waitangi,-43.9535,-176.5597,13.79,76,100,7.60,NZ,49500


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_cities = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = 0.75
)

# Display the map
map_plot_cities 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Choose only cities with a max temperature of between 21 and 27 degrees
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)]
# With a wind speed of less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df['Wind Speed'] < 4.5)]
# And zero cloudiness 
city_data_df = city_data_df.loc[(city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,sirte,31.2089,16.5887,22.56,74,0,2.51,LY,7200
73,73,crane,31.3974,-102.3501,23.48,39,0,2.38,US,-18000
97,97,ubari,26.5921,12.7805,23.56,31,0,2.32,LY,7200
109,109,cabo san lucas,22.8909,-109.9124,25.66,69,0,3.66,MX,-25200
111,111,koungou,-12.7336,45.2042,24.84,94,0,1.03,YT,10800
163,163,maceio,-9.6658,-35.7353,24.69,94,0,4.12,BR,-10800
230,230,saint-pierre,-21.3393,55.4781,22.82,68,0,2.57,RE,14400
244,244,san angelo,31.4638,-100.4370,24.67,73,0,2.06,US,-18000
275,275,zouerate,22.7187,-12.4521,24.34,24,0,3.58,MR,0
281,281,naxos,37.1056,25.3764,22.95,73,0,0.45,GR,10800


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 
                         'Lng', 'Lat', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
42,sirte,LY,16.5887,31.2089,74,
73,crane,US,-102.3501,31.3974,39,
97,ubari,LY,12.7805,26.5921,31,
109,cabo san lucas,MX,-109.9124,22.8909,69,
111,koungou,YT,45.2042,-12.7336,94,
163,maceio,BR,-35.7353,-9.6658,94,
230,saint-pierre,RE,55.4781,-21.3393,68,
244,san angelo,US,-100.4370,31.4638,73,
275,zouerate,MR,-12.4521,22.7187,24,
281,naxos,GR,25.3764,37.1056,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    params["categories"] = 'accommodation.hotel'
    params["apiKey"] = geoapify_key

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        name = name_address["features"][0]["properties"]["name"]
        print(name)
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
فندق المدينة-سرت
sirte - nearest hotel: فندق المدينة-سرت
crane - nearest hotel: No hotel found
فندق اوباري
ubari - nearest hotel: فندق اوباري
Comfort Rooms
cabo san lucas - nearest hotel: Comfort Rooms
Hôtel Trévani
koungou - nearest hotel: Hôtel Trévani
Maceió Hostel e Pousada
maceio - nearest hotel: Maceió Hostel e Pousada
Tropic Hotel
saint-pierre - nearest hotel: Tropic Hotel
Hilton Hotel
san angelo - nearest hotel: Hilton Hotel
فندق تيرس زمور
zouerate - nearest hotel: فندق تيرس زمور
Chateau Zevgoli
naxos - nearest hotel: Chateau Zevgoli
فندق براك السياحي
brak - nearest hotel: فندق براك السياحي
ホテルウラガ
shimoda - nearest hotel: ホテルウラガ
Blue House Hotel
marsa alam - nearest hotel: Blue House Hotel
Oakwood Suites
andrews - nearest hotel: Oakwood Suites
nichinan - nearest hotel: No hotel found
بوناقة
illizi - nearest hotel: بوناقة
JW Marriott Marco Island Beach Resort
marco island - nearest hotel: JW Marriott Marco Island Beach Resort
Appartement Piton Saint Leu
pit

,City,Country,Lng,Lat,Humidity,Hotel Name
42,sirte,LY,16.5887,31.2089,74,فندق المدينة-سرت
73,crane,US,-102.3501,31.3974,39,No hotel found
97,ubari,LY,12.7805,26.5921,31,فندق اوباري
109,cabo san lucas,MX,-109.9124,22.8909,69,Comfort Rooms
111,koungou,YT,45.2042,-12.7336,94,Hôtel Trévani
163,maceio,BR,-35.7353,-9.6658,94,Maceió Hostel e Pousada
230,saint-pierre,RE,55.4781,-21.3393,68,Tropic Hotel
244,san angelo,US,-100.4370,31.4638,73,Hilton Hotel
275,zouerate,MR,-12.4521,22.7187,24,فندق تيرس زمور
281,naxos,GR,25.3764,37.1056,73,Chateau Zevgoli


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = 0.75,
    hover_cols = ['Hotel Name', 'Country']
)

# Display the map
map_plot_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)